[![imagenes/pythonista.png](imagenes/pythonista.png)](https://pythonista.io)

## Creación de documentos PDF a partir de código HTML.

El sitio https://wkhtmltopdf.org/ ofrece una herramienta capaz de transformar código HTML en documentos PDF.
El paquete [pdfkit](https://pypi.python.org/pypi/pdfkit) es una implementación de *wkhtmltopdf* para Python.

### Restricciones. 
El paquete *wkhtmltopdf* requiere del uso de un servidor X y la biblioteca Qt. En el caso de Windows, los binarios incluyen lo necesario para poder ejecutar la herramienta.

Del mismo modo, para que *pdfkit* pueda funcionar, es necesario que *wkhtmltopdf* se encuentre en una ruta conocida, por lo que en el caso de Windows, es necesario añadir esta ruta a la variable de entorno *PATH*.

Los sistemas GNU/Linux para escritorio generalmente vienen con un servidor X habilitado para el uso de entornos gráficos. Sin embargo, es muy probable que un servidor GNU/Linux no cuente con un servidor X instalado.

## Instalación de *wkhtml* junto con un servidor virtual de X en CentOS 7.
La máquina virtual que fue descargada del sitio http://pythonista.io no cuenta con un servidor X, por lo que es necesario instalarlo al igual que el paquete *wkhtmltopdf*.

### Instalación y configuración del servidor X virtual.
El servidor X virtual o *xvfb* por las siglas en inglés de "X Virtual FrameBuffer" es una variante de un servidor X que realiza las mismas opraciones que uno convencional, pero no se despliega en pantalla. Se utiliza para realizar opraciones que requieren de un entrono gráfico en un sistema GNU/Linux.

#### Instalación del paquete.

En este caso se utilizará el gestor de paquetes, *yum*.

In [ ]:
!sudo yum install xorg-x11-server-Xvfb -y

#### Creación del archivo de definición de servicio.
Una vez instalado, se define el servicio añadiendo como superusuario el archivo [*xvfb.service*](/src/xvfb.service) en */etc/systemd/system/* con el siguiente código:

``` bash
[Unit]
Description=Virtual Frame Buffer X Server
After=network.target

[Service]
ExecStart=/usr/bin/Xvfb :42 -screen 0 1024x768x24 -ac +extension GLX +render -noreset

[Install]
WantedBy=multi-user.target
```

In [ ]:
!sudo cp src/xvfb.service /etc/systemd/system/

#### Habilitación y puesta en marcha del servicio.
CentOS 7 utiliza el comando *systemctl* para gestionar servicios.

In [ ]:
!sudo systemctl enable xvfb

In [ ]:
!sudo systemctl start xvfb

Para comprobar que el servidor está corriendo ejecute lo siguiiente:

In [ ]:
!sudo systemctl status xvfb

**Nota:** Este procedimiento sólo se requiere hace una vez. Una vez instalado, configurado y habilitado, el servicio iniciará junto con el servidor.

### Instalación y configuración de *wkhtmltopdf*.

#### Instalación del paquete con *yum*.

In [ ]:
!sudo yum install wkhtmltopdf -y

#### Creación de un script que permita acceder al servidor virtual.

Es necesario que wkthmltopdf pueda acceder al servidor virtual, por lo que se debe de crear un script que incluya al comando *xvfb-run* el cual le indica al sistema que la instrucción hará uso de un servidor X virtual.

El script [src/wkhtmltopdf.sh]([src/wkhtmltopdf.sh]) contiene el siguiente código:

``` bash
#!/bin/bash
xvfb-run -a --server-args="-screen 0, 1024x768x24" /bin/wkhtmltopdf -q $*
```

El script en cuestión debe de contar con permisos de ejecución, lo cual se realzia con el siguiente comando. 

In [ ]:
!chmod +x src/wkhtmltopdf.sh

#### Ejecución del script.

**Nota:** Este es un ejemplo de ejecución en la línea de comandos. No es código en Python. 

La siguiente celda ejecutará el script [src/wkhtmltopdf.sh]([src/wkhtmltopdf.sh]) 
* Se coenctará a http://google/com'.
* El contenido será convertido a PDF y guarado en [google.pdf](google.pdf).

In [ ]:
!src/wkhtmltopdf.sh http://google.com google.pdf

## Instalación de *pdfkit*.

In [ ]:
!pip install pdfkit

In [ ]:
import pdfkit

In [ ]:
help(pdfkit)

### El objeto *pdfkit.configuration*.
Para poder utilizar *pdfkit* es necesario que *wkhtmltopdf* esté instalado y que la ruta donde se encuentra el ejecutable sea conocida.

El objeto *pdfkit.configuration* permite ingresar diversos parámetros de configuración y en estew caso se puede indicar dónde se encuentra el archivo ejecutable de *wkhtmltopdf* con la siguiente sintaxis:

```python
pdfkit.configuration(wkhtmltopdf='<ruta>')
```
En caso de que el ejecutable de *wkhtmltopdf* sea accesible desde el sistema, no es necesario utilizar este objeto.

**Ejemplo:** 

En este caso, el ejecutable que utilizará *pdfkit* corresponde al script localizado en [src/wkhtmltopdf.sh](src/wkhtmltopdf.sh)

In [ ]:
conf = pdfkit.configuration(wkhtmltopdf='src/wkhtmltopdf.sh')

### Funciones de *pdfkit* dependiendo la fuente de los datos.

In [ ]:
pdfkit.from_url('http://coder.mx', 'coder.pdf', configuration=conf)

<p style="text-align: center"><a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/80x15.png" /></a><br />Esta obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Licencia Creative Commons Atribución 4.0 Internacional</a>.</p>
<p style="text-align: center">&copy; José Luis Chiquete Valdivieso. 2018.</p>